# MISATA-IPF v2: Production-Grade Statistical Fidelity

This is the **definitive** implementation. No compromises.

## What We're Fixing
The previous IPF implementation had weak marginal matching (60%). This version:
1. Uses **exact empirical sampling** for marginals
2. Uses **Gaussian copula** for correlation preservation
3. Applies **causal layer** for target variable

## Target Metrics
- Marginal Similarity: **95%+**
- Correlation Similarity: **95%+**
- TSTR Ratio: **90%+**

In [ ]:
!pip install -q numpy pandas scikit-learn matplotlib seaborn scipy

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

print("Setup complete.")

## Part 1: Load and Prepare Data

In [ ]:
# Load Adult Census
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
           'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
           'hours_per_week', 'native_country', 'income']

df_raw = pd.read_csv(url, names=columns, na_values=' ?', skipinitialspace=True)
df_raw = df_raw.dropna().reset_index(drop=True)
df_raw['income'] = (df_raw['income'] == '>50K').astype(int)

# Column types
categorical_cols = ['workclass', 'education', 'marital_status', 'occupation', 
                    'relationship', 'race', 'sex', 'native_country']
numerical_cols = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']

# Encode
df = df_raw.copy()
encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le

# Split
X = df.drop('income', axis=1)
y = df['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
train_df = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)

print(f"Dataset: {len(df_raw):,} rows")
print(f"Train: {len(train_df):,}, Test: {len(X_test):,}")
print(f"Income distribution: {train_df['income'].mean():.1%} high income")

## Part 2: Production IPF Synthesizer

In [ ]:
class ProductionIPFSynthesizer:
    """
    Production-grade IPF-based synthesizer.
    
    Key innovations:
    1. Exact marginal matching via empirical resampling
    2. Correlation preservation via Gaussian copula
    3. Causal target modeling
    """
    
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.fitted = False
        
    def fit(self, df: pd.DataFrame, target_col: str = 'income'):
        """Learn distributions from training data."""
        self.columns = list(df.columns)
        self.target_col = target_col
        self.n_train = len(df)
        
        # Store raw training data for resampling
        self.train_data = df.copy()
        
        # Learn marginal CDFs for each column
        self.marginals = {}
        for col in self.columns:
            values = df[col].values
            self.marginals[col] = {
                'values': np.sort(np.unique(values)),
                'all_values': values,  # Keep all for resampling
                'mean': np.mean(values),
                'std': np.std(values)
            }
        
        # Learn correlation structure via copula
        # Convert to uniform marginals via rank transform
        uniform_df = df.copy()
        for col in self.columns:
            uniform_df[col] = stats.rankdata(df[col]) / (len(df) + 1)
        
        # Convert to normal space
        normal_df = uniform_df.apply(lambda x: stats.norm.ppf(np.clip(x, 0.001, 0.999)))
        
        # Compute and fix correlation matrix
        corr_matrix = normal_df.corr().values
        corr_matrix = np.nan_to_num(corr_matrix, nan=0.0)
        np.fill_diagonal(corr_matrix, 1.0)
        
        # Ensure positive definite
        eigvals, eigvecs = np.linalg.eigh(corr_matrix)
        eigvals = np.maximum(eigvals, 1e-6)
        corr_matrix = eigvecs @ np.diag(eigvals) @ eigvecs.T
        corr_matrix = (corr_matrix + corr_matrix.T) / 2
        np.fill_diagonal(corr_matrix, 1.0)
        
        self.corr_matrix = corr_matrix
        self.cholesky = np.linalg.cholesky(corr_matrix)
        
        # Learn causal model for target
        feature_cols = [c for c in self.columns if c != target_col]
        X_causal = df[feature_cols]
        y_causal = df[target_col]
        
        self.causal_model = GradientBoostingClassifier(
            n_estimators=50, max_depth=4, random_state=self.random_state
        )
        self.causal_model.fit(X_causal, y_causal)
        self.feature_cols = feature_cols
        self.target_rate = y_causal.mean()
        
        self.fitted = True
        return self
    
    def sample(self, n_samples: int) -> pd.DataFrame:
        """Generate synthetic samples."""
        if not self.fitted:
            raise ValueError("Call fit() first")
        
        rng = np.random.default_rng(self.random_state)
        
        # Method: Correlated uniform sampling + quantile transform
        
        # 1. Generate correlated standard normals
        z = rng.standard_normal((n_samples, len(self.columns)))
        correlated_z = z @ self.cholesky.T
        
        # 2. Convert to uniform [0, 1]
        uniform = stats.norm.cdf(correlated_z)
        uniform = np.clip(uniform, 0.001, 0.999)
        
        # 3. Transform each column using empirical quantiles
        synthetic_data = {}
        for i, col in enumerate(self.columns):
            if col == self.target_col:
                continue  # Handle target separately
            
            # Get sorted training values
            sorted_vals = np.sort(self.marginals[col]['all_values'])
            n_vals = len(sorted_vals)
            
            # Compute quantile positions
            positions = np.linspace(0, 1, n_vals)
            
            # Interpolate
            synthetic_vals = np.interp(uniform[:, i], positions, sorted_vals)
            
            # Round categorical/integer columns
            if col in ['age', 'fnlwgt', 'education_num', 'capital_gain', 
                       'capital_loss', 'hours_per_week'] or \
               col in ['workclass', 'education', 'marital_status', 'occupation',
                       'relationship', 'race', 'sex', 'native_country']:
                synthetic_vals = np.round(synthetic_vals).astype(int)
                # Clip to valid range
                min_val = self.marginals[col]['values'].min()
                max_val = self.marginals[col]['values'].max()
                synthetic_vals = np.clip(synthetic_vals, min_val, max_val)
            
            synthetic_data[col] = synthetic_vals
        
        # 4. Generate target using causal model
        synth_features = pd.DataFrame({c: synthetic_data[c] for c in self.feature_cols})
        target_probs = self.causal_model.predict_proba(synth_features)[:, 1]
        
        # Calibrate to match training rate
        threshold = np.percentile(target_probs, (1 - self.target_rate) * 100)
        synthetic_data[self.target_col] = (target_probs >= threshold).astype(int)
        
        # Create DataFrame with correct column order
        return pd.DataFrame(synthetic_data)[self.columns]


# Fit synthesizer
print("Fitting Production IPF Synthesizer...")
start = time.time()
synth = ProductionIPFSynthesizer(random_state=42)
synth.fit(train_df, target_col='income')
fit_time = time.time() - start
print(f"Fitted in {fit_time:.2f}s")

In [ ]:
# Generate synthetic data
print("\nGenerating synthetic data...")
n_synthetic = len(train_df)

start = time.time()
df_synth = synth.sample(n_synthetic)
gen_time = time.time() - start

print(f"Generated {len(df_synth):,} rows in {gen_time:.3f}s")
print(f"Throughput: {len(df_synth)/gen_time:,.0f} rows/sec")
print(f"\nIncome distribution:")
print(f"  Real:      {train_df['income'].mean():.2%}")
print(f"  Synthetic: {df_synth['income'].mean():.2%}")

## Part 3: Comprehensive Evaluation

In [ ]:
def evaluate_comprehensive(real_df: pd.DataFrame, synth_df: pd.DataFrame):
    """Comprehensive statistical fidelity evaluation."""
    results = {}
    
    # 1. Marginal similarity (KS statistic)
    ks_scores = []
    for col in real_df.columns:
        stat, _ = stats.ks_2samp(real_df[col], synth_df[col])
        ks_scores.append(1 - stat)  # Convert to similarity
    results['marginal_similarity'] = np.mean(ks_scores)
    
    # 2. Per-column marginal analysis
    print("\nPer-column marginal similarity (1 - KS statistic):")
    for col in real_df.columns:
        stat, _ = stats.ks_2samp(real_df[col], synth_df[col])
        print(f"  {col}: {1-stat:.2%}")
    
    # 3. Correlation preservation
    num_cols = real_df.select_dtypes(include=[np.number]).columns
    real_corr = real_df[num_cols].corr().values.flatten()
    synth_corr = synth_df[num_cols].corr().values.flatten()
    mask = ~(np.isnan(real_corr) | np.isnan(synth_corr))
    results['correlation_similarity'] = np.corrcoef(real_corr[mask], synth_corr[mask])[0, 1]
    
    # 4. Mean/Std preservation
    mean_scores = []
    std_scores = []
    for col in num_cols:
        real_mean, synth_mean = real_df[col].mean(), synth_df[col].mean()
        real_std, synth_std = real_df[col].std(), synth_df[col].std()
        
        if abs(real_mean) > 0:
            mean_scores.append(1 - min(abs(real_mean - synth_mean) / abs(real_mean), 1))
        if abs(real_std) > 0:
            std_scores.append(1 - min(abs(real_std - synth_std) / abs(real_std), 1))
    
    results['mean_preservation'] = np.mean(mean_scores) if mean_scores else 0
    results['std_preservation'] = np.mean(std_scores) if std_scores else 0
    
    # 5. Overall fidelity
    results['overall_fidelity'] = np.mean([
        results['marginal_similarity'],
        results['correlation_similarity'],
        results['mean_preservation'],
        results['std_preservation']
    ])
    
    return results


# Evaluate
print("=" * 70)
print("STATISTICAL FIDELITY EVALUATION")
print("=" * 70)

fidelity = evaluate_comprehensive(train_df, df_synth)

print(f"\n{'='*70}")
print("SUMMARY")
print(f"{'='*70}")
print(f"Marginal Similarity:    {fidelity['marginal_similarity']:.2%}")
print(f"Correlation Similarity: {fidelity['correlation_similarity']:.2%}")
print(f"Mean Preservation:      {fidelity['mean_preservation']:.2%}")
print(f"Std Preservation:       {fidelity['std_preservation']:.2%}")
print(f"\nOVERALL FIDELITY:       {fidelity['overall_fidelity']:.2%}")
print(f"\nComparison:")
print(f"  Previous MISATA: 54%")
print(f"  Previous IPF v1: 72.6%")
print(f"  Current IPF v2:  {fidelity['overall_fidelity']:.1%}")

## Part 4: TSTR Evaluation

In [ ]:
def evaluate_tstr(df_train_real, df_train_synth, X_test, y_test):
    """Comprehensive TSTR evaluation."""
    results = {}
    
    # TRTR (baseline)
    X_real = df_train_real.drop('income', axis=1)
    y_real = df_train_real['income']
    
    model_real = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    model_real.fit(X_real, y_real)
    y_pred_real = model_real.predict(X_test)
    y_prob_real = model_real.predict_proba(X_test)[:, 1]
    
    results['real'] = {
        'accuracy': accuracy_score(y_test, y_pred_real),
        'roc_auc': roc_auc_score(y_test, y_prob_real),
        'f1': f1_score(y_test, y_pred_real)
    }
    
    # TSTR (synthetic)
    X_synth = df_train_synth.drop('income', axis=1)
    y_synth = df_train_synth['income']
    
    model_synth = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    model_synth.fit(X_synth, y_synth)
    y_pred_synth = model_synth.predict(X_test)
    y_prob_synth = model_synth.predict_proba(X_test)[:, 1]
    
    results['synth'] = {
        'accuracy': accuracy_score(y_test, y_pred_synth),
        'roc_auc': roc_auc_score(y_test, y_prob_synth),
        'f1': f1_score(y_test, y_pred_synth)
    }
    
    results['tstr_ratio'] = results['synth']['roc_auc'] / results['real']['roc_auc']
    
    return results


print("\n" + "=" * 70)
print("TSTR EVALUATION")
print("=" * 70)

tstr = evaluate_tstr(train_df, df_synth, X_test, y_test)

print(f"\nReal (TRTR):")
print(f"  Accuracy: {tstr['real']['accuracy']:.4f}")
print(f"  ROC-AUC:  {tstr['real']['roc_auc']:.4f}")
print(f"  F1:       {tstr['real']['f1']:.4f}")

print(f"\nSynthetic (TSTR):")
print(f"  Accuracy: {tstr['synth']['accuracy']:.4f}")
print(f"  ROC-AUC:  {tstr['synth']['roc_auc']:.4f}")
print(f"  F1:       {tstr['synth']['f1']:.4f}")

print(f"\nTSTR RATIO: {tstr['tstr_ratio']:.2%}")

## Part 5: Visualizations

In [ ]:
# Distribution comparison
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

compare_cols = ['age', 'education_num', 'hours_per_week', 'capital_gain', 'income', 'fnlwgt']

for ax, col in zip(axes.flat, compare_cols):
    if col == 'income':
        x = np.arange(2)
        real_counts = train_df[col].value_counts(normalize=True).sort_index()
        synth_counts = df_synth[col].value_counts(normalize=True).sort_index()
        width = 0.35
        ax.bar(x - width/2, real_counts.values, width, label='Real', alpha=0.8, color='steelblue')
        ax.bar(x + width/2, synth_counts.values, width, label='MISATA-IPF', alpha=0.8, color='coral')
        ax.set_xticks(x)
        ax.set_xticklabels(['<=50K', '>50K'])
    else:
        ax.hist(train_df[col], bins=30, alpha=0.6, label='Real', density=True, color='steelblue')
        ax.hist(df_synth[col], bins=30, alpha=0.6, label='MISATA-IPF', density=True, color='coral')
    
    ax.set_xlabel(col, fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title(f'{col} Distribution', fontsize=12, fontweight='bold')
    ax.legend()

plt.tight_layout()
plt.savefig('misata_ipf_v2_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved misata_ipf_v2_distributions.png")

In [ ]:
# Correlation heatmaps
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

key_cols = ['age', 'education_num', 'hours_per_week', 'capital_gain', 'income']

real_corr = train_df[key_cols].corr()
synth_corr = df_synth[key_cols].corr()

sns.heatmap(real_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0, 
            ax=axes[0], vmin=-1, vmax=1, square=True)
axes[0].set_title('Real Data Correlations', fontsize=12, fontweight='bold')

sns.heatmap(synth_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            ax=axes[1], vmin=-1, vmax=1, square=True)
axes[1].set_title('MISATA-IPF Correlations', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('misata_ipf_v2_correlations.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved misata_ipf_v2_correlations.png")

In [ ]:
# Save all results
results_summary = {
    'method': 'MISATA-IPF-v2',
    'statistical_fidelity': fidelity['overall_fidelity'],
    'marginal_similarity': fidelity['marginal_similarity'],
    'correlation_similarity': fidelity['correlation_similarity'],
    'mean_preservation': fidelity['mean_preservation'],
    'std_preservation': fidelity['std_preservation'],
    'tstr_roc_auc': tstr['synth']['roc_auc'],
    'tstr_f1': tstr['synth']['f1'],
    'tstr_ratio': tstr['tstr_ratio'],
    'generation_time': gen_time,
    'fit_time': fit_time,
    'rows_per_second': n_synthetic / gen_time
}

pd.DataFrame([results_summary]).to_csv('misata_ipf_v2_results.csv', index=False)

print("\n" + "=" * 70)
print("EXPERIMENT COMPLETE")
print("=" * 70)
print(f"\nFinal Metrics:")
print(f"  Statistical Fidelity: {fidelity['overall_fidelity']:.1%}")
print(f"  TSTR Ratio:           {tstr['tstr_ratio']:.1%}")
print(f"  Speed:                {n_synthetic/gen_time:,.0f} rows/sec")
print(f"\nFiles saved:")
print(f"  - misata_ipf_v2_distributions.png")
print(f"  - misata_ipf_v2_correlations.png")
print(f"  - misata_ipf_v2_results.csv")